In [11]:
import PyPDF2
import re
import pandas as pd
import math

def get_bookmarks(outlines, pdf, parent_position_in_bookmark_tree):
    headers = ['position in bookmark tree',
                'original pdf bookmark title (outline.title)',
                'original pdf bookmark page number',
                'corrected pdf bookmark page number',
                'extracted title number',
                'extracted title text',
                'first 20 characters after title',
                'nb_keywords for title search']

    df_content = pd.DataFrame(columns=headers,
                            dtype="string")

    def get_bookmark_content(outline, pdf): # il faut améliorer le code pour pouvoir faire avanter dans la page et pas commencer le get_destination_page_number au debut. reutiliser la derniere page aussi
        if pdf.get_destination_page_number(outline)>=0:
            try:

                # Autres approches:
                #   1/ utiliser calcul de disatnce entre mots
                #   2/ faire simplement "on garde les 6 premier mots du outline.title"
                #   3/

                # il faut traiter les AC separement
                # use word distance

                repattern_title_number_ac = 'AC 2[13759]'+\
                                            '('+\
                                                '\.[0-9]{1,5}[A-Z]?\.?[\s]{1,3}§'+\
                                                    '|'+\
                                                '(?= MG [0-9]{1,2}\.?)'+\
                                            ')' 
                repattern_title_text_ac = '('+\
                                            '2[13759]\.[0-9]{1,5}\.?'+\
                                            ' (\(A[A-Za-z 0-9-]*\))?' +\
                                                    '|'+\
                                            'MG [0-9]{1,2}\.?'+\
                                            ')'      
                                                                                                                         
                repattern_title_prefix = '(VOLUME|Volume|Section|SECTION|section|PART|SUBPART|CHAPTER|chapter|APPENDIX)+[\s]{1,2}'
                repattern_greek = '[ivx]{2,3}|[IVX]{2,3}'
                repattern_title_number_basic = '[\(]?('+repattern_greek+'|[1-9][0-9]{0,3}|[A-Za-z])'
                repattern_title_delimiter = '(-|\)|\.)'

                repattern_title_number ='('+\
                                            repattern_title_number_ac+\
                                        '|'+\
                                            '('+repattern_title_prefix+')?'+\
                                            '('+\
                                                '(-|\)|\.)'.join([repattern_title_number_basic, repattern_title_number_basic, '[ ]?'+repattern_title_number_basic, repattern_title_number_basic])+\
                                            '|'+\
                                                '(-|\)|\.)'.join([repattern_title_number_basic, repattern_title_number_basic, '[ ]?'+repattern_title_number_basic])+\
                                            '|'+\
                                                '(-|\)|\.)'.join([repattern_title_number_basic, repattern_title_number_basic])+\
                                            '|'+\
                                                '(-|\)|\.)'.join([repattern_title_number_basic])+\
                                            ')'+\
                                        ')'+\
                                        '[\s]{0,2}(-|\.)?[\s]{0,2}' # yava (-|\.) avant chaque repattern_title_number_basic
                
                repattern_before_title = '(?<=( |\n))'
                
                for offset in [0,-1,-2,1,2]:                
                    line_with_title = None
                    nb_keywords = 5
                    while line_with_title is None and nb_keywords>0:
                        outline_title_cleaned = re.sub('[^a-zA-Z ,/\d\s\.:-]', '.', outline.title)

                        title_shortened_variable_spaces = re.sub('[ ]+', ' ', outline_title_cleaned).split(' ')[:nb_keywords]
                        title_shortened_variable_spaces = title_shortened_variable_spaces[:2]+\
                                                        [keyword[:2]+'-?'.join(keyword[2:]) for keyword in title_shortened_variable_spaces[2:]]
                        title_shortened_variable_spaces = '[ ]*'.join(title_shortened_variable_spaces)

                        repattern_title_line = '('+repattern_before_title+repattern_title_number+')?'+\
                                                title_shortened_variable_spaces+\
                                                '.*(\n|\r|$)'
                        try:
                            line_with_title = re.search(repattern_title_line,
                                                        pdf.pages[max(0,pdf.get_destination_page_number(outline)+offset)].extract_text())
                        except:
                            pass                        

                        if line_with_title is not None:
                            break
                        else:
                            nb_keywords = nb_keywords - 1

                    if line_with_title is not None:
                        break
                    
                sentence_around_title = pdf.pages[max(0,
                                                      pdf.get_destination_page_number(outline)+offset)].extract_text()[line_with_title.span()[0]:\
                                                                                                                        max(line_with_title.span()[1],\
                                                                                                                            line_with_title.span()[0]+len(outline.title))]
                title_number = re.search(repattern_title_number, sentence_around_title)

                repattern_title_text = '('+repattern_title_text_ac+')?'+\
                                        '[A-Za-z,/\s0-1\(\)\[\]]{3,120}(\.|\:|\?|\n|\r|$)?'# [\s]{1,3}
                if title_number is not None:
                    title_text = re.search(repattern_title_text, 
                                           sentence_around_title[len(title_number.group(0)):])
                    string_after_title = title_text.string[title_text.span()[1]:
                                                            (title_text.span()[1]+30)]

                    bookmark_content = [outline.title,
                                        str(pdf.get_destination_page_number(outline)),
                                        str(pdf.get_destination_page_number(outline)+offset),
                                        title_number.group(0),
                                        title_text.group(0),
                                        string_after_title,
                                        str(nb_keywords)
                                        ]
                    pass
                else:
                    title_text = re.search(repattern_title_text, sentence_around_title[len(title_number.group(0)):])
                    string_after_title = title_text.string[title_text.span()[1]:
                                                           (title_text.span()[1]+30)]

                    bookmark_content = [outline.title,
                                        str(pdf.get_destination_page_number(outline)),
                                        str(pdf.get_destination_page_number(outline)+offset),
                                        '',
                                        title_text.group(0),
                                        string_after_title,
                                        str(nb_keywords)
                                        ]
                    pass
            except:
                bookmark_content = [outline.title,
                                    pdf.get_destination_page_number(outline),
                                    '',
                                    '',
                                    '',
                                    '',
                                    ''
                                    ]
                pass
        else:
            bookmark_content = [outline.title,
                                pdf.get_destination_page_number(outline),
                                '',
                                '',
                                '',
                                '',
                                ''
                                ]
            pass

        
        return bookmark_content

    for n, outline in enumerate(outlines):
        if parent_position_in_bookmark_tree=='':
            current_position_in_bookmark_tree = str(n)
        else:
            current_position_in_bookmark_tree = parent_position_in_bookmark_tree+'>'+str(n)

        if type(outline) == PyPDF2.generic._data_structures.Destination:
            df_thisbookmark_content = get_bookmark_content(outline, pdf)
            df_thisbookmark_content = [current_position_in_bookmark_tree]+df_thisbookmark_content

            headers = ['position in bookmark tree',
                        'original pdf bookmark title (outline.title)',
                        'original pdf bookmark page number',
                        'corrected pdf bookmark page number',
                        'extracted title number',
                        'extracted title text',
                        'first 20 characters after title']

            try:
                df_thisbookmark_content = pd.DataFrame(dict(zip(headers, df_thisbookmark_content)),
                                                    dtype="string", 
                                                    index=[0])
            except UnicodeDecodeError:
                df_thisbookmark_content[1] = df_thisbookmark_content[1].decode('utf-8','ignore')+' (Python Exception: title UnicodeDecodeError)'
                df_thisbookmark_content = pd.DataFrame(dict(zip(headers, df_thisbookmark_content)),
                                                    dtype="string", 
                                                    index=[0])

            df_content = pd.concat([df_content, 
                                    df_thisbookmark_content], 
                                    ignore_index=True)
        elif type(outline) == list:
            extracted_content = get_bookmarks(outline, pdf, current_position_in_bookmark_tree)

            df_content = pd.concat([df_content, 
                                    extracted_content], 
                                    ignore_index=True)
        else:
            raise Exception
        
        # if n>5:
        #     break
        
    return df_content

# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__083AD53732DC0D4686258347005037DF.pdf") # AC 43-13
# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__99C827DB9BAAC81B86256B4500596C4E.pdf") # AC 43-13
# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__C7CCE9FCA6D7E34786257D41004C3E63.pdf") # AC 29
# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\ORDER_8900.1\pdf\Current__DRSDOCID166392477020230614171412.pdf") # Order

# df_content = get_bookmarks(outlines = drs_doc.outline[8:], pdf = drs_doc, parent_position_in_bookmark_tree='') # [8:]

In [13]:
import os
import glob
import pandas as pd
import PyPDF2

drs_doc_types = ['AC',
                 'ORDER_8300.10',
                 'ORDER_8400.10',
                 'ORDER_8700.1',
                 'ORDER_8740.1',
                 'ORDER_8900.1',
                 'ORDERS']

df_all_content = None

for drs_doc_type in drs_doc_types:
    df_index = glob.glob(os.path.join(os.getcwd(), 'data', 'DRS', 'index',drs_doc_type+'*.parquet'))[-1]
    df_index = pd.read_parquet(df_index)

    for index, row in df_index.iterrows():

        matching_file = glob.glob(os.path.join(os.getcwd(), 'data', 'DRS', 'raw data', drs_doc_type, 'pdf','*'+row['documentGuid']+'*.pdf'))
        if len(matching_file)>0:
            if row['drs:status']=='Current':
                print(row['drs:docID']+': '+row['documentGuid'])
                drs_doc = PyPDF2.PdfReader(matching_file[0])
                df_content = get_bookmarks(outlines = drs_doc.outline, pdf = drs_doc, parent_position_in_bookmark_tree='')

                if len(df_content)>0:                    
                    df_content.insert(loc=0, column='documentGuid', value=row['documentGuid'])
                    if df_all_content is not None:
                        df_all_content = pd.concat([df_all_content, 
                                                    df_content], 
                                                    ignore_index=True)
                    else:
                        df_all_content = df_content
                    print('   -> '+str(df_all_content.shape))

                
df_all_content.to_parquet(os.path.join(os.getcwd(), 
                                       'data', 
                                       'DRS', 
                                       'extracted_text_for_'+'-'.join(drs_doc_types)+'.parquet'))

AC 20-153: C2DDC3C8DF201C7386257038006AEECB
   -> (227, 9)
AC 20-153A: CE0FC068CF7D7340862577A5005CF4E4
AC 20-153B: 70E9F099CD3194E586257FA30056FE36


In [10]:
# import PyPDF2
# import re
# import pandas as pd

# def get_bookmarks(outlines, pdf, parent_position_in_bookmark_tree):
#     df_content = pd.DataFrame(columns=['position in bookmark tree',
#                                     'original pdf bookmark title (outline.title)',
#                                     'original pdf bookmark page number',
#                                     'corrected pdf bookmark page number',
#                                     'extracted title number',
#                                     'extracted title text',
#                                     'first 20 characters after title'],
#                             dtype="string")

#     def get_bookmark_content(outline, pdf): # il faut améliorer le code pour pouvoir faire avanter dans la page et pas commencer le get_destination_page_number au debut. reutiliser la derniere page aussi
#         if pdf.get_destination_page_number(outline)>=0:
#             try:
#                 repattern_title_ac = 'AC 2[0-9]\.[0-9]{1,5}\.{0,1} (\(A[A-Za-z 0-9-]\))?'
#                 repattern_title_prefix = 'VOLUME|Volume|Section|SECTION|section|PART|SUBPART|CHAPTER|chapter'
#                 repattern_title_number_basic = '[\(]?([A-Z]{1,1}|[a-z]{1,1}|[0-9]{1,4})[\)\.]?'
#                 repattern_title_number ='('+repattern_title_ac+')?'+\
#                                         '('+repattern_title_prefix+')?'+\
#                                         '( |^)'+\
#                                         repattern_title_number_basic+\
#                                         '((-|\.)'+repattern_title_number_basic+')?'+\
#                                         '((-|\.)'+repattern_title_number_basic+')?'+\
#                                         '((-|\.)[ ]?'+repattern_title_number_basic+')?'+\
#                                         '((-|\.)'+repattern_title_number_basic+')?'+\
#                                         '[\.]?[ ]{0,3}'
                
#                 for offset in [0,-1,-2,1,2]:                
#                     line_with_title = None
#                     nb_keywords = 4
#                     while line_with_title is None and nb_keywords>0:
#                         outline_title_cleaned = re.sub('[^a-zA-Z,\d\s\.:-]', '.', outline.title)

#                         title_shortened_variable_spaces = '[ ]+'.join(re.sub('[ ]+', ' ', outline_title_cleaned).split(' ')[:nb_keywords])
#                         repattern_title_line = '('+repattern_title_number+')?'+\
#                                                 title_shortened_variable_spaces+\
#                                                 '.*(\n|\r|$)' # (^| )?
#                         try:
#                             line_with_title = re.search(repattern_title_line,
#                                                         pdf.pages[max(0,pdf.get_destination_page_number(outline)+offset)].extract_text())
#                         except:
#                             pass
#                         nb_keywords = nb_keywords - 1
#                     if line_with_title is not None:
#                         break
                    
#                 sentence_around_title = pdf.pages[max(0,pdf.get_destination_page_number(outline)+offset)].extract_text()[line_with_title.span()[0]:\
#                                                                                                                             max(line_with_title.span()[1],\
#                                                                                                                                 line_with_title.span()[0]+len(outline.title))]
#                 title_number = re.search(repattern_title_number, sentence_around_title)

#                 repattern_title_text = '[§A-Za-z 0-1\(\)\[\]]{3,50}(\.|\:|\?)?'
#                 if title_number is not None:
#                     title_text = re.search(repattern_title_text, sentence_around_title[len(title_number.group(0)):])
#                     string_after_title = title_text.string[title_text.span()[1]:
#                                                             (title_text.span()[1]+30)]

#                     bookmark_content = [outline.title,
#                                         pdf.get_destination_page_number(outline),
#                                         pdf.get_destination_page_number(outline)+offset,
#                                         title_number.group(0),
#                                         title_text.group(0),
#                                         string_after_title
#                                         ]
#                 else:
#                     title_text = re.search(repattern_title_text, sentence_around_title[len(title_number.group(0)):])
#                     string_after_title = title_text.string[title_text.span()[1]:
#                                                             (title_text.span()[1]+30)]

#                     bookmark_content = [outline.title,
#                                         pdf.get_destination_page_number(outline),
#                                         pdf.get_destination_page_number(outline)+offset,
#                                         '',
#                                         title_text.group(0),
#                                         string_after_title
#                                         ]
#             except:
#                 bookmark_content = [outline.title,
#                                     pdf.get_destination_page_number(outline),
#                                     '',
#                                     '',
#                                     '',
#                                     ''
#                                     ]
#         else:
#             bookmark_content = [outline.title,
#                                 pdf.get_destination_page_number(outline),
#                                 '',
#                                 '',
#                                 '',
#                                 ''
#                                 ]
#         # outline.title, [title_number.group(0), title_text.group(0)]
#         return bookmark_content

#     for n, outline in enumerate(outlines):
#         current_position_in_bookmark_tree = parent_position_in_bookmark_tree+'-'+str(n)

#         if type(outline) == PyPDF2.generic._data_structures.Destination:
#             df_thisbookmark_content = get_bookmark_content(outline, pdf)
#             df_thisbookmark_content = [current_position_in_bookmark_tree]+df_thisbookmark_content

#             df_content = pd.concat([df_content, 
#                                     df_thisbookmark_content], 
#                                     ignore_index=True)

#         elif type(outline) == list:
#             extracted_content = get_bookmarks(outline, pdf, current_position_in_bookmark_tree)
#         else:
#             raise Exception
        

# def add_text_to_bookmarks(bookmarks):
#     for bookmark in bookmarks:
#         # get text and regex
#         # append what's in there
#         pass

#     return bookmarks


# # drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__99C827DB9BAAC81B86256B4500596C4E.pdf") # AC 43-13
# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__C7CCE9FCA6D7E34786257D41004C3E63.pdf") # AC 29
# # drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\ORDER_8900.1\pdf\Current__DRSDOCID166392477020230614171412.pdf") # Order

# get_bookmarks(outlines = drs_doc.outline, pdf = drs_doc, parent_position_in_bookmark_tree='')

KeyboardInterrupt: 

In [ ]:
# import PyPDF2
# import re

# extracted_content = [[["list of parent if"],
#                       'id ()',
#                      'original pdf bookmark title (outline.title)',
#                      'original pdf bookmark page number',
#                      'corrected pdf bookmark page number',
#                      'extracted title number',
#                      'extracted title text',
#                      'first 20 characters after title']]

# def get_bookmarks(outlines, pdf, extracted_content, section_prefix):
#     def get_title(outline, pdf): # il faut améliorer le code pour pouvoir faire avanter dans la page et pas commencer le get_destination_page_number au debut. reutiliser la derniere page aussi
#         if pdf.get_destination_page_number(outline)>0:
#             try:
#                 repattern_title_ac = 'AC 2[0-9]\.[0-9]{1,5}\.{0,1} (\(A[A-Za-z 0-9-]\))?'
#                 repattern_title_prefix = 'Section|SECTION|section|SUBPART|CHAPTER|chapter'
#                 repattern_title_number_basic = '[\(]?([A-Z]{1,1}|[a-z]{1,1}|[0-9]{1,4})[\)\.]?'
#                 repattern_title_number ='('+repattern_title_ac+')?'+\
#                                         '('+repattern_title_prefix+')?'+\
#                                         '( |^)'+\
#                                         repattern_title_number_basic+\
#                                         '((-|\.)'+repattern_title_number_basic+')?'+\
#                                         '((-|\.)'+repattern_title_number_basic+')?'+\
#                                         '((-|\.)'+repattern_title_number_basic+')?'+\
#                                         '((-|\.)'+repattern_title_number_basic+')?'+\
#                                         '[\.]?[ ]{1,3}'
                
#                 for offset in [0,-1,-2,1,2]:                
#                     line_with_title = None
#                     nb_keywords = 4
#                     while line_with_title is None and nb_keywords>0:
#                         outline_title_cleaned = re.sub('[^a-zA-Z\d\s:]', '.', outline.title)

#                         title_shortened_variable_spaces = '[ ]+'.join(re.sub('[ ]+', ' ', outline_title_cleaned).split(' ')[:nb_keywords])
#                         repattern_title_line = '('+repattern_title_number+')?'+\
#                                                 title_shortened_variable_spaces+\
#                                                 '.*(\n|\r|$)' # (^| )?
#                         try:
#                             line_with_title = re.search(repattern_title_line,
#                                                         pdf.pages[max(0,pdf.get_destination_page_number(outline)+offset)].extract_text(),
#                                                         re.MULTILINE)
#                         except:
#                             pass
#                         nb_keywords = nb_keywords - 1
#                     if line_with_title is not None:
#                         break
                    
#                 sentence_around_title = pdf.pages[max(0,pdf.get_destination_page_number(outline)+offset)].extract_text()[line_with_title.span()[0]:\
#                                                                                                            max(line_with_title.span()[1],\
#                                                                                                                line_with_title.span()[0]+len(outline.title))]            

#                 title_number = re.search(repattern_title_number, sentence_around_title, re.MULTILINE)

#                 repattern_title_text = '[§A-Za-z 0-1\(\)\[\]]{3,50}(\.|\:|\?)?'
#                 if title_number is not None:
#                     title_text = re.search(repattern_title_text, sentence_around_title[len(title_number.group(0)):], re.MULTILINE)

#                     title = [pdf.get_destination_page_number(outline), 
#                              outline.title,
#                              title_number.group(0), 
#                              title_text.group(0)]
#                 else:
#                     title_text = re.search(repattern_title_text, sentence_around_title[len(title_number.group(0)):], re.MULTILINE)

#                     title = [pdf.get_destination_page_number(outline),
#                              outline.title,
#                              '',
#                              title_text.group(0)]
#             except:
#                 title = [pdf.get_destination_page_number(outline),
#                          outline.title,
#                          '',
#                          '']
#         else:
#             title = [pdf.get_destination_page_number(outline),
#                      outline.title,
#                      '',
#                      '']
#         # outline.title, [title_number.group(0), title_text.group(0)]
#         return title


#     for outline in outlines:
#         if type(outline) == PyPDF2.generic._data_structures.Destination:
#             title = get_title(outline, pdf)
#             if len(title)>0:
#                 '/Count'
#                 # outline['/Title']
#                 # pdf.get_destination_page_number(outline)
#                 # extracted_content.append([])
#                 # if it's the first one -> section_prefix to be updated.
#             else:
#                 extracted_content.append([section_prefix+[''],
#                                          outline.title, 
#                                          [pdf.get_destination_page_number(outline), pdf.get_destination_page_number(outline)],
#                                          ''])

#         elif type(outline) == list:
#             extracted_content = get_bookmarks(outline, pdf, extracted_content, section_prefix)
#         else:
#             raise Exception
        

# def add_text_to_bookmarks(bookmarks):
#     for bookmark in bookmarks:
#         # get text and regex
#         # append what's in there
#         pass

#     return bookmarks


# # drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__99C827DB9BAAC81B86256B4500596C4E.pdf") # AC 43-13
# # drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__C7CCE9FCA6D7E34786257D41004C3E63.pdf") # AC 29
# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\ORDER_8900.1\pdf\Current__DRSDOCID166392477020230614171412.pdf") # Order

# get_bookmarks(outlines = drs_doc.outline, # [12:]
#               pdf = drs_doc,
#               extracted_content = extracted_content,
#               section_prefix = [])

### Patterns to extract FAA paragraphs

* For Useful FAA documents:
  * AC: 
    * REGEX:
    * EXAMPLE: 

AC 29.79.
§ 29.79 (Amendment 29-21) LIMITING HEIGHT-SPEED ENVELOPE.
(For § 29.79 after Amendment 29-38, see section 29.75A of this AC.)
a. Explanation.
(1) The height-speed envelope is normally referred to as the height-velocity (HV)
diagram. It defines an envelope of airspeed and height above the ground from which a
safe power-off or one engine inoperative (OEI) landing cannot be made. The diagram
normally consists of three portions: (a) the level flight (cruise) portion, (b) the takeoff
portion, and (c) the high speed portion (see Figure AC 29.79-1). The high speed portion
is omitted on occasions when it can be shown that the rotorcraft can suffer an engine
failure at low altitude and high speed (up to VH) and make a successful landing, or climb
out on the remaining engine(s).
(2) Engine power considerations are similar to those in previous takeoff and
landing requirements (see sections 29.53, 29.63, and 29.75 of this AC).
(3) The prohibited sections of the HV diagram are separated by the takeoff
corridor. This corridor should be wide enough to consistently permit a takeoff flight path
clear of the HV

-------


C 29-2C
9/30/99
Page B - 36
AC 29.55. § 29.55 (Amendment 29-39) TAKEOFF DECISION POINT:
CATEGORY A.
a. Explanation.
(1) Amendment 29-39 added a new § 29.55 to redefine the TDP (previously
called the CDP) and contained in § 29.53; it further removed the requirement to identify
the TDP by height and airspeed, since height alone or other factors may be more
appropriate. A Category A takeoff typically begins with an acceleration and/or climb
from a hover to TDP. The rule requires that the TDP be defined for the pilot in terms of
no more than two parameters such as an indicated height and airspeed combination.
(2) The definition of the TDP is based on a minimum required total energy
concept. A specific minimum combination of kinetic energy (airspeed) and potential
energy (height) should be attained at the TDP to ensure that a continued takeoff can be
accomplished following the complete failure of one engine. In § 29.55(b), TDP is
required to be defined by no more than two parameters. When using a single
parameter such as time, height, or airspeed as a method of identifying the TDP, the
identification must be combined with a precisely defined takeoff path and crew
procedure to provide the required equivalent level of safety. In addition, it should be
demonstrated that the pilot technique used during the takeoff sequence is easily
repeatable and consistently produces the required energy (i.e., airspeed and height
combination) when the TDP time, height, or airspeed is attained. This condition should
be verified during the flight test program.
b. Procedures. None.
AC 29.59. § 29.59 (Amendment 29-24) TAKEOFF PATH: CATEGORY A.
a. Explanation. The Category A concept limits the rotorcraft takeoff weight such
that if an engine failure occurs at or before the CDP, a safe landing can be made or if
the engine fails at or after the CDP, the takeoff can be continued. The purpose of these
tests is to define the CDP, evaluate the necessary pilot techniques, and determine the
required takeoff area for either alternative. The condition of equal distances for either
stopping or continuing the
-------------

C 29.623. § 29.623 BEARING FACTORS.
a. Explanation.
(1) The rule requires use of a minimum bearing factor in free fit joints to
account for effects of typical relative motion. A minimum value is not specified in the
rule. The factor, appropriate for the application, is applied to the ultimate bearing
strength of the softest material used as a bearing. A definition of free fit (clearance fit) is
noted in Subparagraph b(7) below.
(2) Specific bearing factors are specified by § 29.685(e) for control system
joints subject to angular rotation. These factors are applied to the ultimate bearing
strength of the softest material used as a bearing in the control system. Control
systems ball, roller, or needle bearings are covered by § 29.685(f).
(3) MIL-HDBK-5C, paragraph 8.3, refers to design standards for plain or journal
bearings or bushings. These standards are found in Air Force Systems Command
Design Handbook 2-1, Airframe, Chapters 2 and 6.
b. Procedures.
(1) Control system joint bearings are discussed under paragraph AC 29.685,
§ 29.685 of this document but the bearing factors are noted here for convenience.
Section 29.685(e) requires a 2.0 bearing factor for cable systems and a 3.33 bearing
factor for push-pull systems other than ball and roller bearing systems. The
manufacturer’s static, non-Brinell rating of ball and roller bearings may not be exceeded
as stated in § 29.685(f).
(2) A landing gear pivot, grease lubricated, plain bearing is one example of a
free fit subject to pounding or vibration. A bearing factor of 2.0 may be used or another
factor may be proven for grease lubricated plain bearing or bushing to account for the
anticipated higher loa
--------


1. PURPOSE. This advisory circular (AC) provides airworthiness and operational approval
guidance material for aircraft operators conducting Title 14 of the Code of Federal Regulations
(14 CFR) part 97 Area Navigation (RNAV) Required Navigation Performance (RNP) instrument
approach proced


compliance to a specific rule by use of a method prescribed in this AC without deviation.
2.0 Cancellation. AC 23.1311-1B, Change 1 “Installation of Electronic Display Instrument
Systems in part 23 Airplanes,” dated February 17, 2009, is cancelled.
3.0 Related Regulations and Documents. Some of the following related documents in this
section may not be referenced in this AC, but they provide additional information or guidance for
electronic displays. Please check for the most recent revision of the documents listed.
3.1 Regulatory Sections. These acceptable means of compliance refer to the applicable
sections of 14 CFR, part 23. For airpl

9.3 Powerplant Information.

6.
RADIO CONTROL EQUIPMENT
a .
Operation.
The air-to-ground radio control equipment permits the pilot to
turn on the airfield lights and select any one of the available intensity steps
(normally three).
The intensity is selected by keying the microphone of the
aircraft communication 


CHAPTER 3. FORMAL APPLICATION PHASE
3.1 General. The next step in the 14 CFR part 141 pilot school certification process is the
formal application phase. This chapter will discuss what documentation must be
submitted for consideration, and the appropriate format. For the formal application phase
to begin, the


n order to prevent distortion, allow the
weld to cool completely, then weld the re­
maining end of the replacement tube to the
original tube.
4-98. REPAIRS AT BUILT-IN FUSE­
LAGE FITTINGS. Make splices in accor­
dance with the methods described in para­
graphs 4-86 through 4-92. Repair built-in fu­
selage fittings in the

es, then weld completely around the
end. In order to prevent distortion, allow the
weld to cool completely, then weld the re­
maining end of the replacement tube to the
original tube.
4-98. REPAIRS AT BUILT-IN FUSE­
LAGE FITTINGS. Make splices in accor­
dance with the methods described in para­
graphs 4-86 through 4-92. Repair built-in fu­
selage fitt

  * ADFRAWD: 
    * REGEX:
    * EXAMPLE:



    


  * ADFREAD:
    * REGEX:
    * EXAMPLE:



    


  * ADNPRM:
    * REGEX:
    * EXAMPLE:



    


  * AFS_FFS_UPDATEPUB:
    * REGEX:
    * EXAMPLE:



    


  * AFS_FOCUS_TEAMS:
    * REGEX:
    * EXAMPLE:



    


  * AIRCRAFT_MASTER_SCHEDULE:
    * REGEX:
    * EXAMPLE:



    


  * ALERTS:
    * REGEX:
    * EXAMPLE:

ULTS WERE, NO ABNORMAL WEAR DETECTED. THIS ENGINE TT O/H WAS 57 HOURS. NOTE; THERE WAS A
SIMILAR FAILURE OF AN OTHER ENGINE OF THIS TYPE JUST 3 MONTHS PRIOR TO THIS ONE, THE TTSOH 123 HRS.
WILL UPDATE THIS REPORT WITH A PROBABLE CAUSE AND RECOMMENDATION WHEN MORE INFORMATION
AVAILABLE. (K)
2007FA0000295 CESSNA
CONT
BEARING
WORN
3/20/2007
414A
TSIO520*
TURBOCHARGER
FAILURE OF RT TURBOCHARGER SEAL AND CENTER BEARING CAUSED SMOKE OUT OF EXHAUST AND
SQUEALING NOISE RESULTING IN THE FLIGHT CREW TO FEATHER PROPELLER AND SHUTDOWN THE ENGINE.
CAUSE OF FAILURE DO TO WEAR. (K)
FMOR477K
CESSNA
CONT
UPLOCK HOOK
DISTORTED
4/9/2007
414A
TSIO520*
57412025
MLG
PILOT R
-------------

RCRAFT LT THE FACTORY.
2007FA0000341 CIRRUS
CONT
RELAY
FAILED
4/16/2007
SR22
IO550N
T/E FLAP
DURING A TRAINING FLIGHT, THE FLAPS BECAME STUCK IN THE DOWN POSITION. THE AIRCRAFT RETURNED TO
THE AIRPORT WITHOUT INCIDENT AND THE AIRCRAFT WAS SENT TO MAINTENANCE FOR TROUBLESHOOTING.
MAINTENANCE DISCOVERED THE FLAPS DOWN RELAY FAILED.
2007FA0000337 CIRRUS
CONT
CONTROL SYSTEM
OUT OF RIG
4/18/2007
SR22
IO550N
AILERON SYS
DURI
    
----------

WAS EVIDENT, WITH LITTLE OR NO PATTERN OF HONEYCOMB IN THE ADHESIVE IN/ ON THE INNER
SURFACE MATERIAL(S). NO EVIDENCE OF POST PRODUCTION REPAIRS WAS APPARENT. ATL MIDO WILL
PROCESS FOR POSSIBLE NON-CONFORMANCE. (K)
2007FA0000229 ISRAEL
GARRTT
VALVE
CRACKED
3/8/2007
1124A
TFE731*
653014
RT LWR FUEL CELL
CRACK IN INTERCONNECT VALVE MOTOR HOUSING. FOUND DURING BAG REMOVAL AND REINSTALLATION.
CAUSE OF CRACK, UNKNOWN. (K)
2007FA0000346 LEAR
GARRTT
ADC
MALFUNCTIONED
1/22/2007
35A
TFE731*
702490031304
DURING P

  * AT_JTA:
    * REGEX:
    * EXAMPLE:



    


  * BULLETINS:
    * REGEX:
    * EXAMPLE:



    


  * CFRFRSFAR:
    * REGEX:
    * EXAMPLE:



    


  * CFRFRSFAR:
    * REGEX:
    * EXAMPLE:



    


  * ELOS:
    * REGEX:
    * EXAMPLE:



    


  * FAR:
    * REGEX:
    * EXAMPLE:


SECTION:   Sec. 60.25
Amendment Number: Initial, Effective Date: 10/30/2006

TITLE:   Operation with missing, malfunctioning, or inoperative components.

SECTION RULE:   (a) No person may knowingly use or allow the use of or misrepresent the capability of an FSTD for any maneuver, procedure, or task that is to be accomplished to meet training, evaluation, or flight experience requirements of this chapter for flight crewmember certification or qualification when there is a missing, malfunctioning, or inoperative (MMI) component that is required to be present and correctly operate for the satisfactory completion of that maneuver, procedure, or task.
(b) Each MMI component as described in paragraph (a) of this section, or any MMI component installed and required to operate correctly to meet the current Statement of Qualification, must be repaired or replaced within 30 calendar days, unless otherwise required or authorized by the NSPM.
(c) A list of the current MMI components must be readily available in or adjacent to the FSTD for review by users of the device. Electronic access to this list via an appropriate terminal or display in or adjacent to the FSTD is satisfactory. The discrepancy log may be used to satisfy this requirement provided each currently MMI component is listed in the discrepancy log.

NPRM ACTIONS:   Not Applicable.

FINAL RULE ACTIONS:   Not Applicable.
    


  * FSB_REPORTS:
    * REGEX:
    * EXAMPLE:



    


  * GA_JTA:
    * REGEX:
    * EXAMPLE:



    


  * INFO:
    * REGEX:
    * EXAMPLE:



An InFO contains valuable information for operators that should help them meet certain administrative, regulatory,
or operational requirements with relatively low urgency or impact on safety.
Subject: Air Carrier Safety and Pilot Training (ACSPT) Aviation Rulemaking Committee
(ARC) Charter and Report
Purpose: This InFO describes where the ACSPT ARC charter and report can be found and
where requested responses to the report should be directed.
Discussion: The FAA convened the ACSPT ARC to meet the requirements set forth in Section
204 of Public Law 111-216. The ACSPT ARC has completed its first report and delivered it to
Congress on July 29th, 2011. The ACSPT ARC charter and report may be found at the following
link: http://www.faa.gov/about/committees/rulemaking/.
The ACSPT ARC is required to produce a second report to be delivered to Congress by July 31st,
2012. This report will include prevalence of use of best practices identified in the first report and
provide recommendations for legislative or regulatory action. The ACSPT ARC is requesting air
carrier industry responses to the report delivered to Congress on July 29, 2011, in order to
support the preparation of this second report and asks that all responses are directed to the
following e-mail address no later than January 31, 2012: acspt.arc@gmail.com.
Contact: Questions or comments regarding this InFO should be directed to the Air
Transportation Division, AFS-200, telephone (202) 267-8166. Questions or comments regarding
the AC


-------------

BJECT: Maneuvers Away From Planned Track in Oceanic Airspace.
Purpose: To inform aircraft operators of the need for pilots to coordinate with the appropriate
air traffic facility when they plan a maneuver away from a cleared route.
Background: Improved aircraft tracking and reporting equipment is coming into use both
onboard trans-oceanic aircraft and in the air traffic facilities controlling trans-oceanic routes.
These new technology applications mean that maneuvers from planned track in excess of
standard Strategic Lateral Offset Procedures automatically are reported to air traffic facilities,
sometimes before flight crews are able to contact the facility regarding their maneuvering. At
present, this can be a problem from a control and airspace coordination standpoint. However, as
capacity enhancements, such as reduced aircraft separation, come into use, maneuvers away
from track without coordination become a greater concern.
Discussion: The trans-oceanic flying environment usually involves long range flying that
transits multiple weather systems. It is normal that flight crews would need to maneuver in
oceanic airspace for the sake of flight safety, but they are required to coordinate such maneuvers
with the appropriate air traffic facility. Oceanic traffic density is increasing, taking advantage of
such technology as enhanced navigation, communication, and in particular surveillance. In this
environment, coordination with air traffic facilities is increasingly important.
Recommended action: Directors of safety, directors of operations (parts 121 and 135), training
managers, and pilots should maintain or refresh their awareness of reporting requirements
involving maneuv
    


  * LEGAL_INTERPRETATIONS:
    * REGEX:
    * EXAMPLE:



    


  * MMEL:
    * REGEX:
    * EXAMPLE:



    


  * MMEL_POLICY_LETTERS:
    * REGEX:
    * EXAMPLE:



    


  * NORSEE:
    * REGEX:
    * EXAMPLE:



    


  * NOTICES:
    * REGEX:
    * EXAMPLE:



    


  * NPRM:
    * REGEX:
    * EXAMPLE:



    


  * OPSS_GUIDANCE:
    * REGEX:
    * EXAMPLE:



    


  * ORDER_8300.10:
    * REGEX:
    * EXAMPLE:



    


  * ORDER_8400.10:
    * REGEX:
    * EXAMPLE:



    


  * ORDER_8700.1:
    * REGEX:
    * EXAMPLE:



    


  * ORDER_8740.1:
    * REGEX:
    * EXAMPLE:



    


  * ORDER_8900.1:
    * REGEX:
    * EXAMPLE:





VOLUME 3 GENERAL TECHNICAL ADMINISTRATION
CHAPTER 9 ISSUE A CERTIFICATE OF WAIVER FOR AEROBATIC
COMPETENCY EVALUATION/PRACTICE OPERATIONS
Section 1 Issue a Certificate of Waiver for Aerobatic Competency
Evaluation/Practice Operations
3-236 PROGRAM TRACKING AND REPORTING SUBSYSTEM (PTRS) ACTIVITY
CODES.
A. Issuance of Certificate of Waiver (CoW). 1230.
B. Review/Revise Aerobatic Competency Evaluation/Practice (ACE/P) Operations
Manual and/or Plan of Activities (POA). 1237.
3-237 OBJECTIVE. The objective of this task is to review and issue, or deny, a request for
a Federal Aviation Administration (FAA) Form 7711-1, Certificate of Waiver or Authorization,
for Aerobatic Competency Evaluation/Practice (ACE/P) operations, and to determine if the
applicant has deve





10/4/17
FY18 FIRST QUARTER EDITORIAL UPDATES (SAS) 8900.1 CHG 504
Vol 2 Ch 11 Sec 3
Page 1
VOLUME 2 AIR OPERATOR AND AIR AGENCY CERTIFICATION AND
APPLICATION PROCESS
CHAPTER 11 CERTIFICATION OF A PART 145 REPAIR STATION
Section 3 Safety Assurance System: Phase 3—Design Assessment (SAS Business Process
Modules 2, 3, and 4)
2-1216 GENERAL. The Design Assessment (DA) phase is the part of the certification process
when the applicant’s manuals and other documents are reviewed indepth to ensure compliance
with applicable regulations and conformity to safe operating practices.
NOTE: These documents may include paper representation or electronic format.
Advisory Circular (AC) 145-9, Guide of Developing and Evaluating Repair
Station and Quality Control Manuals, provides guidance to develop a Repair
Station Manual (RSM) and Quality Control Manual (QCM). AC 145-10, Repair
Station Training Program, provides guidance to develop a training program.
A. Organization. An important responsibility of the certification project manager
(CPM) is to organize the certification team’s efforts to promptly review the applicant’s manuals
and other documents. The Schedule of Events (SOE) determines what will be examined and
when. The previously agreed-upon SOE will determine the priority of items to be reviewed and
any additional inspector support or other Federal Aviation Administration (FAA) resources that
will be needed beyond the composition of the basic certification team. The DA phase is an
intensive process and will most likely require additional resources to accomplish necessary tasks
in a timely manner. The certificate



G. Determine if Exceptions Apply to the Operator.
1) Certificate Holders That Do Not Operate in Ground Icing Conditions.
The part 135 ground deicing rule does not apply to a certificate holder that does not operate in
ground icing conditions. This certificate holder is not required to train its pilots or develop
pretakeoff contamination check procedures. Certificate holders that do not operate in ground
icing conditions will have that limitation in their OpSpec A042.
2) Operators Using Only One Pilot in Operations. Single-pilot operators are
not required to comply with the manual and approved training requirements of § 135.21 or
§ 135.341. Therefore, single-pilot operators are not required to have an approved pilot training
program or the additional training required by the part 135 ground deicing rule. However,
single-pilot operators must comply with all of the operational requirements of the part 135
ground deicing rule. Those operational requirements include a pretakeoff contamination check or
an approved alternative procedure to the pretakeoff contamination check described in its
OpSpec. The pilots of these types of operators will need to demonstrate sufficient knowledge to
operate in ground icing conditions during the initial and recurrent flight checks. A single-pilot
operator will have an




  * ORDERS:
    * REGEX:
    * EXAMPLE:

Chapter 6. Designated Mechanic Examiner (DME) and Designated Parachute
Rigger Examiner (DPRE) Program
Section 1. DME—General
1. Additional Requirements for Appointment. The procedures contained in this chapter apply
to inspectors, DMEs, authorized to conduct aviation mechanic or parachute rigger oral and
practical tests and issue Temporary Airman Certificates. DMEs will conduct tests in accordance
with the applicable practical test standards (PTS). Inspectors must carefully consider all of the
following issues before designating or renewing a designee. In addition to meeting the general
designee eligibility requirements in Chapter 2, a designee applicant must meet the following
additional requirements, which will be reviewed by the appointing Federal Aviation
Administration (FAA) office:
a. For DME Appointment.
(1) Evidence of a high level of knowledge and experience in the subject areas required
for aviation mechanic certification in both reciprocating and turbine-engine aircraft.
(2) Previously held a valid aviation mechanic certificate for 5 years with the rating(s) for
which a desig


Section 3-3. Lateral Navigation with
Vertical Guidance (LNAV/VNAV) Final Segment
3-3-1. General. An LNAV/VNAV approach is a vertically-guided approach procedure using
baro-VNAV or WAAS VNAV for the vertical guidance. Obstacle evaluation is based on the
LNAV OEA dimensions, level surface OCS based on ROC, and a sloping baro-VNAV OCS.
ROC adjustments for excessive length of final do not apply.
a. Vertical Path restrictions. The true vertical path provided by baro-VNAV is influenced by
temperature variations; i.e., during periods of cold temperature, the effective glidepath may be
lower than published and during periods of hot weather, the effective glidepath may be higher
than published (see paragraph 3-3-4.a(1) and Order 8260.19 paragraph 4–6–8).
b. Other restrictions. LNAV/VNAV approach procedures at airports where Remote
Altimeter Setting Source (RASS) is in use, helicopter point-in-space (PinS) approaches, or where
the final segment overlies precipitous terrain, must be annotated to indicate the approach is not
authorized for baro-VNAV s






Section 2. Processing an Application for a PMA
4-5. Applicant Responsibilities.
a. Application Letter. The applicant must submit a letter of application to an ACO or
MIDO/CMO, depending on the design approval basis. If the applicant is applying on the basis
of an STC or identicality by licensing agreement, the application will be submitted to the
MIDO/CMO having geographical responsibility for the area in which the applicant’s
manufacturing facil

    
Chapter 5. Technical Standard Order Authorization (Part 21, Subpart O)
Section 1. General Information
5-1. Applicability. Part 21, subpart O, is applicable to a person who desires to manufacture
an article that meets a specific TSO. The TSO authorization system does not apply to articles
produced under a PMA, TC, or PC. The TSO authorization is both an FAA design and
production approval issued to an applicant who has demonstrated the capability to design and
manufacture a specific TSO. The FAA does not separate the State of Design from the State of
Manufacture.
5-2. Privileges. A TSO authorization holder has the privileges specified in subpart O and within
the letter of TSO authorization. In addition, a TSO authorization holder is eligible to have a
qualified employee(s) designated as a DMIR in accordance with the provisions of part 183. The
TSO authorization holder may also be authorized by part 183 to apply for and obtain an ODA.
FAA Orders 8000.95 and 8100.15 contain procedures for the administration of DMIRs and
ODAs, respectively.
5-3. Advising the Applicant. The applicant will be advised that—
a. AC 21-43 sets forth an acceptable means of complying with part 21, subpart O.
Alternative methods and procedures may be



hapter 3. Concerns about Destroyed and Scrapped Aircraft
1. FAA Concerns.
a. Determining whether an aircraft is repairable, destroyed, or scrapped. The
regulations do not set forth specific criteria that can be used to determine whether an aircraft is
repairable or whether it is destroyed or scrapped.
b. Failure of certificate holders to terminate registration. Pursuant to 14 CFR
§ 47.41(a)(2), the registration of an aircraft that is totally destroyed or scrapped is no longer
effective. The owner of the aircraft must notify the FAA by returning the aircraft’s registration
certificate to the FAA’s Aircraft Registration Branch (hereafter, Registry) within 21 days (or by
providing a statement to the FAA if the registration certificate is not available) in accordance
with 14 CFR § 47.41(b). A review of records for aircraft classified as destroyed or scrapped has
indicated that a number of owners may not have returned these certificates to the Registry as
required. Accordingly:
(1) The Registry may not be accurate or up-to-date.
(2) Aircraft r



  * OSR:
    * REGEX:
    * EXAMPLE:



    


  * OSWG:
    * REGEX:
    * EXAMPLE:



    


  * OTHER_EFB_CHECKLISTS:
    * REGEX:
    * EXAMPLE:



    


  * OTHER_EFB_RESEARCH_REPORTS:
    * REGEX:
    * EXAMPLE:



    


  * OTHER_FAA_90_DAY_SAFETY_REVIEW:
    * REGEX:
    * EXAMPLE:



    


  * OTHER_INTERNATIONAL_PUBLICATIONS:
    * REGEX:
    * EXAMPLE:



    


  * OTHER_JOB_AIDS:
    * REGEX:
    * EXAMPLE:



    


  * OTHER_PS_FEDERAL_AVIATION_ACTS:
    * REGEX:
    * EXAMPLE:



    


  * OTHER_PS_HANDBOOKS:
    * REGEX:
    * EXAMPLE:



    


  * OTHER_RCCB:
    * REGEX:
    * EXAMPLE:



    


  * PILOT_QUALIFICATION_CURRICULUM:
    * REGEX:
    * EXAMPLE:



    


  * POLICY:
    * REGEX:
    * EXAMPLE:


Summary
This policy statement addresses the certification of Title 14 Code of Federal Regulations (CFR)
Part 23 aircraft that include a Wireless Local Area Network (WLAN), using Institute of
Electrical and Electronics Engineers (IEEE) 802.11 a/b/g/n protocols. More specifically, the
policy is to clarify the guidance regarding cabin WLAN equipment installation that creates a
Radio Frequency (RF) network and provides connectivity (i.e., internet connection and email
services) through access point(s) to users with IEEE 802.11 compliant Portable Electronic
Devices (PEDs). The primary concern of the subject WLAN certification is the aircraft systems
immunity and compatibility with RF energy from transmitting PEDs (T-PEDs).
Current Regulatory and Advisory Material
The regulations applicable to the installation of WLAN using IEEE 802.11 protocols are 14 Title
CFR §§ 23.1301, 23.1309(a), (b)(1), and 23.1431(b).
The following Advisory Circulars (ACs) have guidance that is relevant to WLAN installation:
 AC 20-168, Certification Guidance for Installation of Non-Essential, Non-Required
Aircraft Cabin Systems & Equipment (CS&E)
 AC 20-164, Designing and Demonstrating Aircraft Tolerance to Portable Electronic
Devices
 AC 23-1309-1E, System Safety Analysis and Assessment for Part 23 Airplanes
2
Background and Relevant Past Practice
1.
Past FAA certification projects involving WLAN systems focused on the downlink of
information from satellites with limited uplink and transmission from the airplane cabin.
The configuration of transmitters and receivers within the cabin was rigorously controlled
and qualified to airplane installation standards.
2.
The current WLAN installation involves the use of several transmitters from various
consumer electronic equipment (e.g., laptop/tablet computers, smart phones, etc.) whose
transmission specifications and failure modes cannot be rigorously analyzed and tested due
to the unregulated use of these devices on an airplane.
3.
New computer interfaces, networks and data protocols have been developed to use wireless
radio-frequency (RF) links to transfer data. This allows communications without hard-
wired connections. One approach uses IEEE standard 802.11a/b/g/n, which collectively is
known as Wi-Fi, for wireless communication. IEEE 802.11b, g, and n use wideband
digital modulation techniques in the 2400 to 2483.5 MHz band. IEEE 802.11a and n uses
wideband digital modulation techniques in the 5250 to 5350 MHz and 5470 to 5825 MHz
frequency bands. Wireless transmitters using IEEE 802.11a, b. g, or n protocol do not
require FCC licenses and are allowed to have 1 watt transmitter power with up to 6 dBi
antenna gain, which results in 4 watts Effective Isotropic Radiated Power (EIRP) in the
US.
4.
Safety issues related to the installation and use of the wireless RF system within the
airplane include:
 Potential interference with avionics systems whose failure conditions are
classified as major or worse, or any required systems
 Operation of PED and a wireless RF system which is not fully built to airborne
equipment standards
 Vulnerability of airplane systems to intentional or spurious emission of RF energy
5.
An issue paper has been used recently to address the certification and compliance concerns
related to WLAN installation and RF field strength resulting from PED usage.
6.
AC 20-164 was published with guidance on demonstrating that aircraft and installed
systems are tolerant to potential interference from portable electronic devices.
Policy
The policy points out the items to be considered, and methods acceptable to the FAA regarding
the aircraft toleranc
    


  * SAFO:
    * REGEX:
    * EXAMPLE:



 ble to air carriers in meeting their statutory duty to provide service with the highest possible degree of safety in the public
interest. Besides the specific action recommended in a SAFO, an alternative action may be as effective in addressing the safety
issue named in the SAFO.
Subject: Title 14 of the Code of Federal Regulations (14 CFR) Part 121 Operators Flap Misconfiguration
Events
Purpose: This SAFO serves to raise awareness of aircraft misconfigurations on takeoff with an emphasis
on flap position.
Background: The Aviation Safety Information Analysis and Sharing (ASIAS) program recently
concluded an analysis of flap misconfiguration events. This analysis included takeoff attempts with flaps
set at zero and flap movement during the takeoff roll.
Discussion: The ASIAS analysis indicates that while flap misconfiguration events on takeoff are rare,
they do exist. Approximately half of the events for misconfiguration during the takeoff roll resulted in
rejected takeoffs. The other takeoffs were continued with the flaps being moved during the takeoff roll.
The configuration of the airplane by the flightcrew should be in accordance with the operational
checklists. It is imperative that flightcrews exercise discipline in the use and the execution of operational
checklists to prevent aircraft misconfigurations on takeoff.
Recommended Action: Directors of Operations, Directors of Safety, Program Managers, Directors of
Training, Training Center Program Managers, Check Pilots, Training Pilots, Chief Pilots and flightcrews
should be familiar with the information contained in this SAFO. They should work together to ensure that
the content of this SAFO is emphasized in their training programs to raise awareness among their
flightcrews of flap misconfiguration events. This is especially important if procedures are being changed
in light of the new Federal Aviation Administration (FAA) de-icing procedures. Operators should review
their own Aviation Safety Action Program (ASAP) and Flight Operational Quality Assurance (FOQA)
data for flap misconfiguration events. Operators are also reminded to conduct an analysis on any change
as part of their Safety Management System (SMS) processes.
Contact: Questions   


  * SAIB:
    * REGEX:
    * EXAMPLE:



    


  * SAS_AXH_DCT:
    * REGEX:
    * EXAMPLE:



    


  * SAS_DCT:
    * REGEX:
    * EXAMPLE:



    


  * SCFINAL:
    * REGEX:
    * EXAMPLE:



    


  * SCPROPOSED:
    * REGEX:
    * EXAMPLE:



    


  * SFAR:
    * REGEX:
    * EXAMPLE:



    


  * TSO:
    * REGEX:
    * EXAMPLE:


Subject: Personnel Parachute Assemblies and Components
1.
PURPOSE. This technical standard order (TSO) is for manufacturers applying for a TSO
authorization (TSOA) or letter of design approval (LODA). In it, we the Federal Aviation
Administration (FAA), tell you what minimum performance standards (MPS) your personnel
parachute assembly and components must first meet for approval and identification with the
applicable TSO marking.
2.
APPLICABILITY. This TSO affects new applications submitted after its effective date.
a. All prior revisions to this TSO are no longer effective. Generally, we will not accept
applications for the previous revision after the effective date of this TSO. We may do so,
however, up to six months after it, if we know that you were working against the prior MPS
before the new change became effective.
b. Personnel parachute assemblies and components approved under a previous TSOA may
still be manufactured under the provisions of its original approval.
3. REQUIREMENTS. New models of personnel parachute assemblies and components
identified and manufactured on or after the effective date of this TSO must meet the MPS
qualification and documentation requirements in Parachute Industry Association (PIA)
Technical Standard 135 TS-135 Revision 1.4 issued April 22, 2010 “Performance Standards for
Personnel Parachute Assemblies and Components” as modified by appendix 1 of this TSO.
a. Functionality. This TSO’s standards apply to equipment intended to be used as a
reserve or emergency parachute.
b. Failure Condition Classifications.
(1) Lose of t
    





g. Deviations. We have provisions for using alternate or equivalent means of compliance
with the criteria in the MPS of this TSO. If you invoke these provisions, you must show that
your equipment maintains an equivalent level of safety. Apply for a deviation pursuant to Title
14 of the Code of Federal Regulations (CFR) 21.618.
4.
MARKING.
a. Mark at least one major component permanently and legibly with all of the information
in 14 CFR 45.15(b). Mark each article according to Equipment Class(es) from Table 1 above.
b. If the article includes software and/or airborne electronic hardware, then the article part
numbering scheme must identify the software and airborne electronic hardware configuration.
The part numbering scheme can use separate, unique part numbers for software, hardware, and
airborne electronic hardware.
c. You may use electronic part marking to identify software or airborne electronic hardware
components by embedding the identification within the hardware component itself (using
software) rather than marking it on the equipment nameplate. If electronic marking is used, it
must be readily access







  * UNAPPROVED_PARTS_NOTIFICATIONS:
    * REGEX:
    * EXAMPLE:



    



  * (6) Training on Records.
  * 21. PASSENGER AND CARGO LOADING PROCEDURES. 
  * (1) Loading, based on aircraft configuration, i.e.,....
  * 1. Purpose.
  * Purpose: This InFO informs air carriers of the availability of the Department of Transportation’s (DOT) National Aviation Resource Manual for Quarantinable Diseases. 
  * SUBJECT: DOT

In [ ]:
import re

faire un ScribdDocument

commence en début de ligne


### Use a Hugging Face or Kaggle model
* 